In [2]:
import numpy as np
import wavload
import librosa
import math        
import pyaudio     
import random
import tflearn
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [3]:
"""Making samples - Mark 2 """

fs = 44100       # sampling rate, Hz, must be integer
duration = 5   # in seconds, may be float

num_samples = 100
freq = np.zeros((num_samples,10))
amp = np.zeros((num_samples,10))
X_data = np.zeros((num_samples,fs*duration))


for i in range (0,num_samples):
    freq1 = random.uniform(16,2000)        # sine frequency, Hz, may be float
    freq2 = 2*freq1
    freq3 = 3*freq1
    freq4 = 4*freq1
    freq5 = 5*freq1
    freq6 = 6*freq1
    freq7 = 7*freq1
    freq8 = 8*freq1
    freq9 = 9*freq1
    freq10 = 10*freq1

    amp1 = random.uniform(0,1)     # range [0.0, 1.0]
    amp2 = random.uniform(0,amp1)
    amp3 = random.uniform(0,amp2)
    amp4 = random.uniform(0,amp3)
    amp5 = random.uniform(0,amp4)
    amp6 = random.uniform(0,amp5)
    amp7 = random.uniform(0,amp6)
    amp8 = random.uniform(0,amp7)
    amp9 = random.uniform(0,amp8)
    amp10 = random.uniform(0,amp9)


    # generate samples, note conversion to float32 array
    sample1 = (amp1*(np.sin(2*np.pi*np.arange(fs*duration)*freq1/fs))).astype(np.float32)
    sample2 = (amp2*(np.sin(2*np.pi*np.arange(fs*duration)*freq2/fs))).astype(np.float32)
    sample3 = (amp3*(np.sin(2*np.pi*np.arange(fs*duration)*freq3/fs))).astype(np.float32)
    sample4 = (amp4*(np.sin(2*np.pi*np.arange(fs*duration)*freq4/fs))).astype(np.float32)
    sample5 = (amp5*(np.sin(2*np.pi*np.arange(fs*duration)*freq5/fs))).astype(np.float32)
    sample6 = (amp6*(np.sin(2*np.pi*np.arange(fs*duration)*freq6/fs))).astype(np.float32)
    sample7 = (amp7*(np.sin(2*np.pi*np.arange(fs*duration)*freq7/fs))).astype(np.float32)
    sample8 = (amp8*(np.sin(2*np.pi*np.arange(fs*duration)*freq8/fs))).astype(np.float32)
    sample9 = (amp9*(np.sin(2*np.pi*np.arange(fs*duration)*freq9/fs))).astype(np.float32)
    sample10 = (amp10*(np.sin(2*np.pi*np.arange(fs*duration)*freq10/fs))).astype(np.float32)

    audio = ((sample1
                 +sample2
                 +sample3
                 +sample4
                 +sample5
                 +sample6
                 +sample7
                 +sample8
                 +sample9
                 +sample10)/10)

    freq[i] = (freq1,freq2,freq3,freq4,freq5,freq6,freq7,freq8,freq9,freq10)    
    amp[i] = (amp1,amp2,amp3,amp4,amp5,amp6,amp7,amp8,amp9,amp10)
    X_data[i] = audio
    #print(audio.shape)
    
# for paFloat32 sample values must be in range [-1.0, 1.0]
#stream = p.open(format=pyaudio.paFloat32,
#                    channels=1,
#                    rate=fs,
#                    output=True)

# play. May repeat with different volume values (if done interactively) 
#stream.write(audio)

#stream.stop_stream()
#stream.close()

#p.terminate()


In [4]:
"""Taking MFCC of audio data"""
MFCC = np.zeros((num_samples,20,431))
for i in range (0,num_samples):
    MFCC[i] = librosa.feature.mfcc(y=X_data[i], sr=fs)
print(MFCC.shape)

(100, 20, 431)


In [5]:
"""Loading parameters as Y data"""
Y_data = np.concatenate((freq , amp), axis=1)
print(Y_data.shape)

(100, 20)


In [6]:
"""Defining important variables for LSTM"""

features_cols = MFCC[0].shape[0]
features_rows = 1
parameter_size = Y_data[0].shape[0]
print(parameter_size)

20


In [31]:
"""LSTM network"""

net = tflearn.input_data([None, features_cols, features_rows])
net = tflearn.lstm(net, 128, dropout=0.8)
net = tflearn.fully_connected(net, parameter_size, activation='relu')
net = tflearn.regression(net, optimizer='adam', learning_rate=0.001,
                         loss='mean_square')
model = tflearn.DNN(net, tensorboard_verbose=0)
model.fit(train_x, train_y, validation_set=(test_x, test_y), show_metric=True,
          batch_size=32, n_epoch=1)

KeyboardInterrupt: 

In [7]:
#X_data = np.expand_dims(X_data, axis = 2)

In [7]:
model = Sequential()
model.add(LSTM(128))
model.add(Dense(units=20, activation='relu'))
model.compile(loss='mean_squared_error',
              optimizer='sgd',
              metrics=['accuracy'])
model.fit(MFCC, Y_data)

W0912 19:46:19.387125 139620787279680 deprecation.py:323] From /home/josephcleveland/Research/venv/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


100/100 [==============================] - 0s 3ms/sample - loss: 28733858.4800 - acc: 0.5000
